In [ ]:

# %pip install imbalanced-learn

In [ ]:
import pandas as pd
import numpy as np

from imblearn.under_sampling import RandomUnderSampler

In [ ]:
!gsutil cp /home/clas_giulia_s/buckets/b1/datasets/competencia_02_fe_v01.parquet /home/clas_giulia_s/datasets/

In [ ]:
base_path = '/home/clas_giulia_s/buckets/b1/'

dataset_path = base_path + 'datasets/'
modelos_path = base_path + 'modelos/'
db_path = base_path + 'db/'
dataset_file = 'competencia_02_fe_v01.parquet'

semillas = [261823, 289871, 379817, 481301, 959723]

data = pd.read_parquet('/home/clas_giulia_s/datasets/' + dataset_file)

In [ ]:
data['clase_binaria'] = 0
data['clase_binaria'] = np.where(data['clase_ternaria'] == 'BAJA+2', 1, 0)

In [ ]:
meses_train = [201906, 201907, 201908, 201909, 201910, 201911, 201912,
               202001, 202002, 202003, 202004, 202005, 202006,
               202007, 202008, 202009, 202010, 202011, 202012,
               202101, 202102, 202103, 202104, 202105, 202106]

data = data[data['foto_mes'].isin(meses_train)]

In [ ]:
data.shape

In [ ]:
proportions = (
    data.groupby('foto_mes')['clase_binaria']
    .value_counts(normalize=True)
    .unstack(fill_value=0)
    .reset_index()
)

proportions.columns = ['foto_mes', 'proportion_0', 'proportion_1']
proportions

In [ ]:
df_subsampled = []

for mes, group in data.groupby("foto_mes"):

    X = group.drop(columns="clase_binaria")
    y = group["clase_binaria"]
    
    # Calculo la proporcion de bajas
    minority_proportion = y.value_counts(normalize=True).get(1, 0)
    print(f'Original proportion:{minority_proportion}')
    
    # voy a incrementar esa proporcion por 10
    new_proportion = minority_proportion * 10

    # Apply RandomUnderSampler with the calculated strategy
    rus = RandomUnderSampler(sampling_strategy=new_proportion, random_state=semillas[0])
    X_res, y_res = rus.fit_resample(X, y)

    # Rebuild the resampled DataFrame for the group
    group_resampled = pd.concat([X_res, y_res], axis=1)
    group_resampled["foto_mes"] = mes  # Add back the 'foto_mes' column

    # Add the resampled DataFrame to the list
    df_subsampled.append(group_resampled)

# Concatenate all resampled groups into a single DataFrame
data = pd.concat(df_subsampled, ignore_index=True)

In [ ]:
data.shape

In [ ]:
proportions = (
    data.groupby('foto_mes')['clase_binaria']
    .value_counts(normalize=True)
    .unstack(fill_value=0)
    .reset_index()
)

proportions.columns = ['foto_mes', 'proportion_0', 'proportion_1']
proportions

In [ ]:
output_file = 'competencia_02_fe_v01_undersampled.parquet'

data.to_parquet('/home/clas_giulia_s/datasets/' + output_file, index=False)

In [ ]:
!gsutil cp /home/clas_giulia_s/datasets/competencia_02_fe_v01_undersampled.parquet /home/clas_giulia_s/buckets/b1/datasets/